In [1]:
from src.SupertypeTools import CGCalcMat, dist_heatmap, SSE, hierarchical_cluster, Silhouette, correlation, Tuning_N, crop_mtx, SSE, triangle2square
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np

In [3]:
Nmin = 1
Nmax = 8

A_order = [
    ['A01_01', 'A26_01', 'A26_02', 'A26_03', 'A29_02', 'A30_01', 'A30_02', 'A30_03', 'A30_04', 'A32_01'],
    ['A02_01', 'A02_02', 'A02_03', 'A02_04', 'A02_05', 'A02_06', 'A02_07', 'A02_14', 'A02_17', 'A68_02', 'A69_01'],
    ['A03_01', 'A11_01', 'A31_01', 'A33_01', 'A33_03', 'A66_01', 'A68_01', 'A74_01'],
    ['A23_01', 'A24_02']
    ]

B_order = [
    ['B07_02','B07_03','B07_05','B15_08','B35_01','B35_03','B42_01','B51_01','B51_02','B51_03','B53_01','B54_01','B55_01','B55_02','B56_01','B67_01','B78_01'],
    ['B08_01','B08_02'],
    ['B14_02','B15_03','B15_09','B15_10','B15_18','B27_02','B27_03','B27_04','B27_05','B27_06','B27_07','B27_09','B38_01','B39_01','B39_02','B39_09','B48_01','B73_01'],
    ['B18_01','B37_01','B40_01','B40_02','B40_06','B44_02','B44_03','B45_01'],
    ['B15_16','B15_17','B57_01','B57_02','B58_01','B58_02'],
    ['B15_01','B15_02','B15_12','B15_13','B46_01','B52_01']
    ]

C_order = [
    ['C01_02','C03_02','C07_02','C08_01','C08_02','C12_02','C14_02','C16_01','C16_04'],
    ['C02_02','C03_07','C03_15','C04_01','C05_01','C06_02','C07_07','C12_04','C12_05','C15_02','C16_02','C17_01','C18_01']
    ]

BA_Mat_all = pd.read_csv("Computed_DistMtx/top10_matrix.csv", index_col=0)
BA_Mat_all = triangle2square(BA_Mat_all)

BA = crop_mtx(BA_Mat_all, A_order, flatten=True)
BB = crop_mtx(BA_Mat_all, B_order, flatten=True)
BC = crop_mtx(BA_Mat_all, C_order, flatten=True)


# SSE_BA, _, _, _ = Tuning_N(crop_mtx(BA_Mat_all, A_order), Nmin, Nmax, None)
# SSE_BB, _, _, _ = Tuning_N(crop_mtx(BA_Mat_all, B_order), Nmin, Nmax, None)
# SSE_BC, _, _, _ = Tuning_N(crop_mtx(BA_Mat_all, C_order), Nmin, Nmax, None)

In [6]:
from sklearn.model_selection import ParameterGrid

SimMtx = "SM_THREAD_NORM"
# Grantham, PMBEC, SM_THREAD_NORM
grid = ParameterGrid({'sigma':[0.075, 0.1, 0.2, 0.3, 0.5],
                    'k':[0.8,1,2,4,8,16]})


for param in grid:

    sigma=param['sigma']
    k=param['k']
    print(sigma, k)

    # ======== HLA-A ===========
    AMat = CGCalcMat("HLA1_models/CG_DAT", SimMtx=SimMtx, AlleleListFile="Dataset_split/ReferencePanel-A.list", sigma=sigma, k=k)
    AMat = triangle2square(AMat)

    SA = crop_mtx(AMat, A_order, flatten=True)
    RA = correlation(SA, BA, show_plot=False)[2]
    _, SSE_SA, _, _ = Tuning_N(AMat, Nmin, Nmax, BA_Mat_all)
    # _, ASSE_BA3, _, _ = Tuning_N(AMat, 3, 3, BA_Mat_all)
    # _, ASSE_BA6, _, _ = Tuning_N(AMat, 6, 6, BA_Mat_all)

    # ========= HLA-B ===========
    BMat = CGCalcMat("HLA1_models/CG_DAT", SimMtx=SimMtx, AlleleListFile="Dataset_split/ReferencePanel-B.list", sigma=sigma,k=k)
    BMat = triangle2square(BMat)
    
    SB = crop_mtx(BMat, B_order, flatten=True)
    RB = correlation(SB, BB, show_plot=False)[2]
    _, SSE_SB, _, _ = Tuning_N(BMat, Nmin, Nmax, BA_Mat_all)
    # _, BSSE_BA3, _, _ = Tuning_N(BMat, 3, 3, BA_Mat_all)
    # _, BSSE_BA6, _, _ = Tuning_N(BMat, 6, 6, BA_Mat_all)

    # ========= HLA-C ===========
    CMat = CGCalcMat("HLA1_models/CG_DAT", SimMtx=SimMtx, AlleleListFile="Dataset_split/ReferencePanel-C.list", sigma=sigma, k=k)
    CMat = triangle2square(CMat)
    
    SC = crop_mtx(CMat, C_order, flatten=True)
    RC = correlation(SC, BC, show_plot=False)[2]
    _, SSE_SC, _, _ = Tuning_N(CMat, Nmin, Nmax, BA_Mat_all)
    # _, CSSE_BA, _, _ = Tuning_N(CMat, 2, 2, BA_Mat_all)

    
    # print(sum(np.array(SSE_SA) - np.array(SSE_BA)), sum(np.array(SSE_SB) - np.array(SSE_BB)), sum(np.array(SSE_SC) - np.array(SSE_BC)))
    print(sum(SSE_SA) + sum(SSE_SB) + sum(SSE_SC))
    # np.add(np.add(SSE_BA, SSE_BB), SSE_BC)
    # print(ASSE_BA3, ASSE_BA6, BSSE_BA3, BSSE_BA6, CSSE_BA)
    print(sum([RA, RB, RC]))

# Doytchinova SSE:

# Sidney SSE:

#6.16953559 3.6466475399999996 16.219613250000002 7.081748650000001 1.9410084

0.075 0.8
171.13458726
1.9199140742609
0.1 0.8
176.2826516
1.9936975222187312
0.2 0.8
164.04689824000002
2.1803117100134353
0.3 0.8
149.99973672
2.2313025129338406
0.5 0.8
152.75868414
2.2173367648828597
0.075 1
185.41315487999998
1.9502445718193866
0.1 1
168.85800951999997
2.0307049048796713
0.2 1
151.06809588000002
2.205979643000902
0.3 1
155.85604007999999
2.2343761720482753
0.5 1
150.83689238
2.2038760852708874
0.075 2
165.56165463999997
2.060429976534158
0.1 2
161.75901692
2.1455157698181244
0.2 2
157.03915863999998
2.2334000170116157
0.3 2
149.99880663
2.209285485335344
0.5 2
158.69862945999998
2.159380083085958
0.075 4
162.67645980000003
2.1683105224039414
0.1 4
149.52771164
2.2207072213508052
0.2 4
154.71909863
2.208669706661038
0.3 4
151.47220481999997
2.17120457590386
0.5 4
153.25539414
2.114568295621111
0.075 8
149.912541
2.2276400750307603
0.1 8
153.46664362
2.2294401959401964
0.2 8
151.47220481999997
2.17428074225507
0.3 8
160.7200528
2.130630114789468
0.5 8
150.1673381199